In [46]:
{'a':'4'}**{'b':'3'}

TypeError: unsupported operand type(s) for ** or pow(): 'dict' and 'dict'

In [37]:
import numpy as np
import plotly.io as pio
import plotly.graph_objects as go
import pandas as pd
import matplotlib as mpl
from http import server
from dash import Dash, html, dcc
import plotly.express as px
import dash_bootstrap_components as dbc
from glob import glob
from dash.dependencies import Input, Output

In [38]:
## Palette
def colorFader(c1,c2,mix=0.0): #fade (linear interpolate) from color c1 (at mix=0) to c2 (mix=1)
    c1=np.array(mpl.colors.to_rgb(c1))
    c2=np.array(mpl.colors.to_rgb(c2))
    return mpl.colors.to_hex((1-mix)*c1 + mix*c2)

#gradiente neon
# c1 = '#f72585'
# c2 = '#4cc9f0'

#gradiente dark RB
# c1 = '#b7094c'
# c2 = '#0091ad'

#gradiente desaturated mint
# c1 = '#cad2c5'
# c2 = '#2f3e46'

#mint gradient
# c1 = '#B9CBC3'
# c2 = '#1a231f'

#Sage and Fern
c1 = '#5a7976'
c2 = '#b0dadd'

#vapor pastel gradient
# c1 = '#70BAFF'
# c2 = '#FF85B1'

##############################################################################à
## Init
# Set variabili iniziali
p_in = 0.5
rep = 5  #repliche, diversi loci indipendenti
gens = 100
xx = np.arange(gens+1)
#template della palette dei grafici
pio.templates['grad'] = go.layout.Template(
    layout=dict(
        colorway=[colorFader(c1, c2, x/rep) for x in range(rep)]
    )
)
############################################################################
#Funzione di estrazione modello Wright-Fisher
def extract(p, size):
    N=size
    randomlist = np.random.rand(2*N)
    newgen_alleles = [1 if x <= p else 0 for x in randomlist]
    # calcolo dei genotipi
    newgentype = [(newgen_alleles[i]+newgen_alleles[i+1]) for i in range(0, (2*N), 2)]
    #calcolo della nuova frequenza
    newp = sum(newgen_alleles)/(2*N)
    return newgen_alleles, newgentype, newp

#Calcolo della frequenza p ad ogni generazione
def calcp_vector(p, gens, size):
    pv = []
    pv.append(p_in)
    for g in range(gens):
        a, b, p = extract(p, size)
        pv.append(p)
        if p == 0 or p == 1: break
    while len(pv) < gens+1:
        pv.append(pv[-1])
    return pv

#matrice di pv per repliche indipendenti
def calcp_matrix(size):
    pm = []
    for r in range(rep+1):
        pv = calcp_vector(p_in, gens,size)
        pm.append(pv)
    return pm


#grafico frequenza contro generazioni
#Qui uso Plotly go
def graph_freq(pm,size):
    fig = go.Figure()
    fig.update_layout(
        yaxis_range=[0, 1],
        template='plotly_white+grad',
        title=dict(
            text='Population Dimension N=%s' %size,
            font=dict(size=16),
        )
    )
    fig.update_xaxes(
        title_text='Generations'
    )
    fig.update_yaxes(
        title_text='Allele Frequency'
    )

    for i in range(rep):
        fig.add_trace(go.Scatter(
            x=xx,
            y=pm[i],
            showlegend=False
        ))
    return fig

#Calcolo deltaP per selezione
def deltaP(p_vec, w_vec, w11, w12, w22):
    qq = 1 - p_vec[-1]
    w_mean = (p_vec[-1]) ** 2 * w11 + 2 * p_vec[-1] * qq * w12 + qq ** 2 * w22
    w_vec.append(w_mean)
    deltap = (p_vec[-1] * qq * (
                (p_vec[-1] * w11 + qq * w12) - (p_vec[-1] * w12 + qq * w22))) / w_mean
    return deltap

#Calcolo della frequenza p ad ogni generazione CON SELEZIONE
def calcp_vector_sel(size, p, gens, w11, w12, w22):
    pv = []
    pv.append(p)
    w_vec = []
    for g in range(gens):
        a, b, p = extract(p, size)
        if p == 0 or p == 1:
            pv.append(p)
            break
        deltap = deltaP(pv,w_vec, w11, w12, w22)
        p = p + deltap
        pv.append(p)
    while len(pv) < gens+1:
        pv.append(pv[-1])
    return pv


def calcp_matrix_sel(size, w11, w12, w22, p0):
    pm = []
    for r in range(rep+1):
        pv = calcp_vector_sel(size, p0, gens, w11, w12, w22)
        pm.append(pv)
    return pm

In [39]:
def plot_drift(string='selection', p0=0.01, w11=1, w12=0.8, w22=0.6):
    size = int(input('Select Size:'))
    if size == None:
        return ''
    if string == 'drift':
        pm = calcp_matrix(size)
    if string == 'selection':
        pm = calcp_matrix_sel(size, w11, w12, w22, p0)
    return graph_freq(pm, size)

In [40]:
plot_drift('drift')